In [91]:
# Created by: Chen Da
# Created on: 20191031

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [93]:
# LR函数
def functionLR(id_data):
    z = intercept + np.dot(coefs, id_data.values[1:-1])
    prob = 1 / (1 + np.exp(-z))
    return prob

In [94]:
# 肺癌预测
def lungCancerPrediction(raw_df, file_name):
    df = raw_df.loc[:, selected_columns] 
    prob_list = list()
    for info in df['id'].values:
        prob = functionLR(df[df['id'] == info].iloc[0, :])
        prob_list.append(prob)
    df_pro = df.loc[:, ['id', 'class']]
    df_pro['prob'] = pd.Series(prob_list)
#     df_healthy = df_pro[df_pro['class'] == 0]
#     df_unhealthy = df_pro[df_pro['class'] == 1]
#     plt.figure(figsize=(12,10))
#     plt.scatter(df_healthy['id'], df_healthy['prob'], color='red')
#     plt.scatter(df_unhealthy['id'], df_unhealthy['prob'], color='skyblue')
#     plt.axhline(y=0.5, color='grey', linestyle='--', alpha=0.6)
#     plt.xticks(rotation=90)
#     plt.grid()
#     plt.show()
    df_pro['class'] = df_pro['class'].apply(lambda x : str(x))
    df_pro['id'] = df_pro['id'].apply(lambda x : str(x)+'_')
    df_pro = df_pro.sort_values(by='class')
    fig = px.scatter(df_pro, x='id', y='prob',
                     color='class', template='ggplot2', color_discrete_sequence=['green', '#EE0000'],
                     color_discrete_map={'0': 'green', '1': '#EE0000'})
    fig.add_trace(go.Scatter(x=[df_pro['id'].iloc[0, ], df_pro['id'].iloc[-1,]], y=[0.5, 0.5], 
                             mode='lines', 
                             line=dict(color='grey', dash='dash'),
                             showlegend=False))
    fig.update_layout(width=1200, height=800)
#     fig.to_image('C:/Users/pc/Desktop/output.png')
#     py.io.write_image(fig, '/Users/chenda/Desktop/%s' % file_name + '.png')
    fig.show()
    return df_pro

In [95]:
file_name = input("请输入原始数据的名称：")
data_path = 'C:/Users/pc/OneDrive/PLTTECH/Project/20191205_lung_cancer/rawdata/%s' % file_name + '.xlsx'
df = pd.read_excel(data_path)
df.iloc[:, 1:-1] = df.iloc[:, 1:-1].multiply(100)
df.head()

# 模型所用变量
selected_columns = ['id', 'CD3', 'CD4', 'CD8', 'CD45', 'IGD', 'CD11b', 'CD14', 'CD19',
       'CD20', 'CD27', 'CD33', 'CD39', 'CD86', 'CD94', 'CXCR5', 'gdTCR',
       'CD57', 'CD11c', 'tbet', 'CD16', 'CD127', 'granzyme_B', 'HLA_DR',
       'CD161', 'CD56', 'CD197', 'CD68', 'CD24', 'CD28', 'ki67', 'PD1',
       'CD278', 'CCR6', 'CD123', 'CD25', 'FOXP3', 'CD274', 'CD152', 'CD85j',
       'CD183', 'CD38', 'class']


# 模型系数
coefs = [0.052284986882095735,
 -0.028630797108041072,
 0.0719086726027658,
 -0.02692928239868485,
 -0.028234877415327125,
 -0.013480136957200867,
 0.007978267794720021,
 -0.0379658224396516,
 -0.04050972628103438,
 0.09176454153172824,
 -0.03523314739708188,
 -0.17318265955797219,
 -0.045076284814311404,
 -0.013659996061472989,
 -0.023055214482174402,
 -0.03285800944574266,
 0.03015199448669844,
 0.014911373072056247,
 0.0006409013765632889,
 0.0972999630734454,
 -0.02187400465136366,
 -0.12908014353211558,
 -0.016961373206987023,
 0.017415380102301384,
 0.036219650768987456,
 -0.04603655258459527,
 0.09189456875720244,
 -0.040538782717477725,
 0.04311843953295698,
 0.04919531801743276,
 -0.029037238815441964,
 0.060262690510385214,
 0.05821294611702925,
 0.028065712015949297,
 0.023931379052632627,
 -0.04882907553939195,
 -0.03705243393533725,
 0.01085403037978642,
 0.01459589990011545,
 0.16146110305742867,
 -0.07368430219786196]

intercept = -5.5548236129141465e-05

plot_df = lungCancerPrediction(df, file_name)

请输入原始数据的名称：marker肺结节


In [96]:
# plot_df.to_excel('C:/Users/pc/Desktop/%s.xlsx'%file_name, index=False)

In [97]:
plot_df['pre_label'] = plot_df['prob'].apply(lambda x: round(x))
plot_df['class'] = plot_df['class'].apply(lambda x: int(x))

In [98]:
pre_0_right = plot_df[np.logical_and(plot_df['class']==0, plot_df['pre_label']==0)].shape[0]
pre_0_wrong = plot_df[np.logical_and(plot_df['class']==0, plot_df['pre_label']==1)].shape[0]
pre_1_right = plot_df[np.logical_and(plot_df['class']==1, plot_df['pre_label']==1)].shape[0]
pre_1_wrong = plot_df[np.logical_and(plot_df['class']==1, plot_df['pre_label']==0)].shape[0]
print('总体准确率为%s' % ((pre_0_right+pre_1_right)/plot_df.shape[0]))

总体准确率为0.9230769230769231


In [99]:
pre_0_right, pre_0_wrong, pre_1_right, pre_1_wrong

(12, 1, 0, 0)

In [100]:
print('灵敏度为%s' % (pre_1_right/(pre_1_right+pre_1_wrong)))       # 正确判断病人的概率
print('特异度为%s' % (pre_0_right/(pre_0_right+pre_0_wrong)))       # 正确判断非病人的概率

ZeroDivisionError: division by zero